# UTILS

In [ ]:
def convert_to_world_point(x, y, depth_map, 
                           image_sensor_width=32.0*1e-3,
                           image_sensor_height = 18.0*1e-3,
                           focal_length = 10.0*1e-3):
    """
    Returns the real world coordinates of pixels (x, y) 
    given the depth map
    
    Input:
        - x, y: int, pixel coordinates
        - depth_map: np.array of size (W, H)
    
    Output:
        - world_x, world_y, world_z: tuple of int
    """
    image_center_x = 1024 / 2.0 #depth_map.shape[1] / 2.0
    image_center_y = 512 / 2.0 # depth_map.shape[0] / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (image_sensor_width / 1024)
    sensor_z = px_z * (image_sensor_height / 512)
    
    d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / focal_length
    world_z = (world_y * sensor_z) / focal_length
    return (world_x, world_y, world_z)

In [ ]:
image_sensor_width = 32.0*1e-3
image_sensor_height = 18.0*1e-3
focal_length = 10.0*1e-3

# GROUND TRUTH

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from utils import crop_and_mask
from obb import OBB
import cv2

In [ ]:
# noise = np.zeros((512, 1024), np.uint8)
# cv2.randn(noise, np.array(7), np.ones(1)*5.0)
# print(noise)
# plt.imshow(noise)
# plt.colorbar()
# plt.show()

In [ ]:
left_path = '/root/data/blender/blender_test/Image0028_L.png'
right_path = '/root/data/blender/blender_test/Image0028_R.png'
ground_truth_depth_path = '/root/data/blender/blender_test/true_depth.npy'

In [ ]:
# get ground truth mask + segmentation
mask = crop_and_mask(Image.open(left_path))
ground_truth_depth = np.load(ground_truth_depth_path) #.transpose()

In [ ]:
# visualize both
plt.imshow(mask)
plt.show()

plt.imshow(ground_truth_depth)
plt.show()

mdepth = ground_truth_depth*mask
plt.imshow(mdepth>10)
# plt.colorbar()
plt.show()

In [ ]:
# # quick hack because the crop and mask function is not optimal
x, y = np.nonzero(mdepth>10)
for (i,j) in zip(x,y):
    mask[i,j] = 0

In [ ]:
mdepth = ground_truth_depth*mask
plt.imshow(mdepth)
plt.colorbar()
plt.show()

In [ ]:
# calculate ground truth length
y, x = np.nonzero(ground_truth_depth*mask)
wx, wy, wz = convert_to_world_point(x, y, ground_truth_depth)
cloud = []
for (i0, j0, k0) in zip(wx, wy, wz):
    cloud.append([i0, j0, k0])
obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in cloud])
true_obb_points = np.array(obb.points)
true_length = np.linalg.norm(true_obb_points[0] - true_obb_points[1])
print('True length: {}'.format(true_length))

# ERODE THE MASK

In [ ]:
lengths = []
errors = []
for s in range(0, 11):
    if s > 0:
        kernel = np.ones((s, s))
        eroded_mask = cv2.erode(mask, kernel)
    else:
        eroded_mask = mask
    
    
    total = eroded_mask + mask
    intersection = np.count_nonzero(total[total==2])
    union = np.count_nonzero(total[total>0])
    iou = intersection*100 / union
    # print('Intersection over Union: {}'.format(intersection/float(union)))
    # print('Error: {}'.format(1 - intersection/float(union)))
    errors.append(100 - iou)
    
    
    # calculate ground truth length
    y, x = np.nonzero(ground_truth_depth*eroded_mask)
    wx, wy, wz = convert_to_world_point(x, y, ground_truth_depth)
    cloud = []
    for (i0, j0, k0) in zip(wx, wy, wz):
        cloud.append([i0, j0, k0])
    obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in cloud])
    true_obb_points = np.array(obb.points)
    pred_length = np.linalg.norm(true_obb_points[0] - true_obb_points[1])
    
    # print('True length: {}'.format(pred_length))
    relative_error = np.abs(pred_length-true_length) / true_length
    lengths.append(relative_error)

In [ ]:
true_obb_points.shape

In [ ]:
plt.plot(errors, lengths)
plt.xlabel('Segmentation error (erosion)')
plt.ylabel('Length relative error')
plt.show()

# DILATE THE MASK

In [ ]:
lengths = []
errors = []
for s in range(0, 11):
    if s > 0:
        kernel = np.ones((s, s))
        eroded_mask = cv2.dilate(mask, kernel)
    else:
        eroded_mask = mask
    
    
    total = eroded_mask + mask
    intersection = np.count_nonzero(total[total==2])
    union = np.count_nonzero(total[total>0])
    iou = intersection*100 / union
    # print('Intersection over Union: {}'.format(intersection/float(union)))
    # print('Error: {}'.format(1 - intersection/float(union)))
    errors.append(100 - iou)
    
    
    # calculate ground truth length
    y, x = np.nonzero(ground_truth_depth*eroded_mask)
    wx, wy, wz = convert_to_world_point(x, y, ground_truth_depth)
    cloud = []
    for (i0, j0, k0) in zip(wx, wy, wz):
        cloud.append([i0, j0, k0])
    obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in cloud])
    true_obb_points = np.array(obb.points)
    pred_length = np.linalg.norm(true_obb_points[0] - true_obb_points[1])
    # print('True length: {}'.format(pred_length))
    relative_error = np.abs(pred_length-true_length) / true_length
    lengths.append(relative_error)

In [ ]:
plt.plot(errors, lengths)
plt.xlabel('Segmentation error (dilatation)')
plt.ylabel('Length relative error')
plt.show()

# ADD NOISE TO DEPTH MAP

In [ ]:
import copy

In [ ]:
lengths = []
errors = []
for s in range(0, 11):
#     total = eroded_mask + mask
#     intersection = np.count_nonzero(total[total==2])
#     union = np.count_nonzero(total[total>0])
#     iou = intersection*100 / union
#     # print('Intersection over Union: {}'.format(intersection/float(union)))
#     # print('Error: {}'.format(1 - intersection/float(union)))
#     errors.append(100 - iou)
    new_depth = copy.deepcopy(mdepth)
    if s > 0:
        mean_depth = mdepth.sum() / np.count_nonzero(mdepth)
        

        # creat some noise
        noise = np.zeros((512, 1024), np.uint8)
        cv2.randn(noise, np.array(0), np.ones(1)*s)
        new_depth += noise
        
    
    depth_relative_error = np.nanmean(np.abs(new_depth*mask-mdepth) / mdepth)
    print(depth_relative_error)
    errors.append(depth_relative_error)
    # calculate ground truth length
    y, x = np.nonzero(new_depth*mask)
    wx, wy, wz = convert_to_world_point(x, y, new_depth)
    cloud = []
    for (i0, j0, k0) in zip(wx, wy, wz):
        cloud.append([i0, j0, k0])
    obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in cloud])
    true_obb_points = np.array(obb.points)
    pred_length = np.linalg.norm(true_obb_points[0] - true_obb_points[1])
    # print('True length: {}'.format(pred_length))
    relative_error = np.abs(pred_length-true_length) / true_length
    lengths.append(relative_error)

In [ ]:
plt.plot(errors, lengths)
plt.xlabel('Depth map error (noise)')
plt.ylabel('Length relative error')
plt.show()

# BOTH

In [ ]:
lengths = np.zeros((22, 22))
errors_mask = []
errors_depth = []
all_masks = []
all_depth = []

for size in range(0, 22):
    if size > 0:
        kernel = np.ones((size, size))
        new_mask = cv2.dilate(mask, kernel)
    else:
        new_mask = mask
            
    total = new_mask + mask
    intersection = np.count_nonzero(total[total==2])
    union = np.count_nonzero(total[total>0])
    iou = intersection*100 / union
    # print('Intersection over Union: {}'.format(intersection/float(union)))
    # print('Error: {}'.format(1 - intersection/float(union)))
    errors_mask.append(100 - iou)
    all_masks.append(new_mask)
    
for s in range(0, 22):

    new_depth = copy.deepcopy(mdepth)
    if s > 0:
        mean_depth = mdepth.sum() / np.count_nonzero(mdepth)


        # creat some noise
        noise = np.zeros((512, 1024), np.uint8)
        cv2.randn(noise, np.array(0), np.ones(1)*s)
        new_depth += noise
        
    all_depth.append(new_depth)
    depth_relative_error = np.nanmean(np.abs(new_depth*mask-mdepth) / mdepth)
    # print(depth_relative_error)
    errors_depth.append(depth_relative_error*100)
    
for (i,new_mask) in enumerate(all_masks):
    for (j, new_depth) in enumerate(all_depth):

        # calculate ground truth length
        y, x = np.nonzero(new_depth*new_mask)
        wx, wy, wz = convert_to_world_point(x, y, new_depth)
        cloud = []
        for (i0, j0, k0) in zip(wx, wy, wz):
            cloud.append([i0, j0, k0])
        obb, eigen_vectors = OBB.build_from_points([(p[0], p[1], p[2]) for p in cloud])
        true_obb_points = np.array(obb.points)
        pred_length = np.linalg.norm(true_obb_points[0] - true_obb_points[1])
        
        
        # print(pred_length)
        # print('True length: {}'.format(pred_length))
        relative_error = np.abs(pred_length-true_length) / true_length
        lengths[j, i] = relative_error*100

In [ ]:
len(errors_depth), len(errors_mask), len(lengths)

In [ ]:
errors_mask

In [ ]:
plt.contourf(errors_mask, errors_depth, lengths, levels=range(0, int(lengths.max())+50, 50))
plt.xlabel('Segmentation error (%)')
plt.ylabel('Depth relative error (%)')
plt.title('DILATATION')
plt.colorbar()

In [ ]:
plt.imshow(length)

In [ ]:
len(errors_mask)

In [ ]:
len(errors_depth)

In [ ]:
lengths.shape

In [ ]:
s = 1